In [1]:
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import sys
sys.path.append(".")
import os
# os.environ["OPENAI_API_KEY"]="sk-xxxxxx"

from agent.agents import agent_factory

In [2]:
def show_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [3]:
import requests

img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg"
image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")


prompt="""task: I want to get in the car."""


In [4]:
agent = agent_factory("DetLLM")
plan_result = agent.plan(prompt, image)
plan_result

PlanResult(success=False,
 error_message='No objects were detected in the image.',
 plan_raw=None,
 masks=None,
 prompt=None,
 plan_code=None,
 annotated_image=None,
 info_dict={'objects_to_detect': ['refrigerator', 'oven', 'microwave', 'toaster', 'blender', 'coffee maker', 'dishwasher', 'pot', 'pan', 'cutting board', 'knife', 'spoon', 'fork', 'plate', 'bowl', 'cup', 'coaster', 'glass', 'kettle', 'paper towel holder', 'trash can', 'food storage container', 'sofa', 'coffee table', 'television', 'bookshelf', 'armchair', 'floor lamp', 'rug', 'picture frame', 'curtain', 'blanket', 'vase', 'indoor plant', 'remote control', 'candle', 'wall art', 'clock', 'magazine rack']})

In [5]:
agent = agent_factory("SegVLM")
plan_result = agent.plan(prompt, image)
plan_result

PlanResult(success=True,
 error_message=None,
 plan_raw="To get into the car, we will need to open the car door. I will identify the part of the image that represents the car's door handle and will instruct the robot to open it.\n\n```python\n# The door handle is likely represented by the mask over the car's handle\nopen(regions[14])\n```\n\nAfter running the above command, the robot should attempt to open the car door by the handle, allowing you to get inside the car.",
 masks=[{'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])}],
 prompt='task: I want to get in the car.',
 plan_code="\n# The door handle is likely represented by the mask over the car's handle\nopen(regions[0]